In [ ]:
%%time
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
import xgboost as xgb

from sklearn import preprocessing
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.decomposition import PCA
from math import sqrt
from scipy import stats
from scipy.stats import norm, skew #for some statistics

In [ ]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, average_precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
from tqdm import tqdm_notebook
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# download flair library #
import torch
!pip install flair
import flair

In [ ]:
os.listdir('../input/nlp_datac2476d7/NLP_Data')

In [ ]:
train = pd.read_csv('../input/predictive_data32f5357/Predictive_Data/train_file.csv')
test = pd.read_csv('../input/predictive_data32f5357/Predictive_Data/test_file.csv')
subm = pd.read_csv('../input/predictive_data32f5357/Predictive_Data/sample_submission.csv')

In [ ]:
train = train.drop(['UsageClass','CheckoutType','CheckoutYear','CheckoutMonth'],axis=1)
test = test.drop(['UsageClass','CheckoutType','CheckoutYear','CheckoutMonth'],axis=1)

In [ ]:
train.head()
test.head()

In [ ]:
train.shape
test.shape

In [ ]:
print('class count in numbers: ')
train['MaterialType'].value_counts()
print('percentage of class count : ')
train['MaterialType'].value_counts()/train.shape[0] * 100

In [ ]:
# Number of NaNs in each rows
train.isnull().sum(axis=1).head(5)
train.isnull().sum(axis=0)

In [ ]:
test.isnull().sum(axis=0)

In [ ]:
train.nunique()
test.nunique()

In [ ]:
set(train['PublicationYear'])

In [ ]:
train['PublicationYear'].replace(np.nan,"0000",inplace=True)
test['PublicationYear'].replace(np.nan,"0000",inplace=True)

In [ ]:
train['Subjects'].replace(np.nan,"",inplace=True)
test['Subjects'].replace(np.nan,"",inplace=True)

In [ ]:
import re
def matc(a):
    return re.findall(r"[0-9]{4}",a)
train['PublicationYear'] = train['PublicationYear'].apply(lambda x: matc(x))
test['PublicationYear'] = test['PublicationYear'].apply(lambda x: matc(x))

In [ ]:
train['combined'] = train['Title'] + ' . ' + train['Subjects'] + ' . ' + train['Creator'] + ' . ' + train['Publisher']
test['combined'] = test['Title'] + ' . ' + test['Subjects'] + ' . ' + test['Creator'] + ' . ' + test['Publisher']

In [ ]:
def ext(a):
    #print(a)
    if len(a)==0:
        return 0
    #print(int(a[0]))
    return int(a[0])
train['PublicationYear'] = train['PublicationYear'].apply(lambda x:ext(x))
test['PublicationYear'] = test['PublicationYear'].apply(lambda x:ext(x))

In [ ]:
# set(train['MaterialType'])

train['MaterialType'].replace("BOOK",0,inplace=True)
train['MaterialType'].replace("CR",1,inplace=True)
train['MaterialType'].replace("MIXED",2,inplace=True)
train['MaterialType'].replace("MUSIC",3,inplace=True)
train['MaterialType'].replace("SOUNDCASS",4,inplace=True)
train['MaterialType'].replace("SOUNDDISC",5,inplace=True)
train['MaterialType'].replace("VIDEOCASS",6,inplace=True)
train['MaterialType'].replace("VIDEODISC",7,inplace=True)

In [ ]:
train['Publisher'].replace(np.nan,"",inplace=True)
test['Publisher'].replace(np.nan,"",inplace=True)

train['Creator'].replace(np.nan,"",inplace=True)
test['Creator'].replace(np.nan,"",inplace=True)

## Tfidf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# TF-IDF feature matrix
train1 = tfidf_vectorizer.fit_transform(train['combined'])
test1 = tfidf_vectorizer.fit_transform(test['combined'])

In [ ]:
train['combined'].shape
test['combined'].shape

np.hstack([train['combined'].values,test['combined'].values]).shape

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# create the transform
vectorizer = CountVectorizer(stop_words="english", analyzer='word', 
                            ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None)
# tokenize and build vocab
vectorizer.fit(np.hstack([train['combined'].values,test['combined'].values]))
train2 = vectorizer.transform(train['combined'])
test2 = vectorizer.transform(test['combined'])
print(train2.shape)
print(type(train2))

In [ ]:
train2
test2

In [ ]:
csr_matrix([train['Checkouts']]).T

In [ ]:
from scipy.sparse import hstack, coo_matrix, csr_matrix,vstack
train3 = hstack((train1, train2))

test3 = hstack((test1, test2))

In [ ]:
train3
test3

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# splitting data into training and validation set
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train3, train['MaterialType'], random_state=42, test_size=0.2)

# lreg = LogisticRegression()
lreg = lgb.LGBMRegressor()
lreg.fit(xtrain_bow, ytrain) # training the model

prediction = lreg.predict(xvalid_bow) # predicting on the validation set
# prediction_int = prediction[:,1] >= 0.3 # if prediction is greater than or equal to 0.3 than 1 else 0
prediction_int = prediction.astype(np.int)

f1_score(yvalid, prediction_int,average='micro') # calculating f1 score

test_pred = lreg.predict(test3)

## Logistic

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


# splitting data into training and validation set
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train3, train['MaterialType'], random_state=42, test_size=0.2)

lreg = LogisticRegression()
# lreg = lgb.LGBMRegressor()
lreg.fit(xtrain_bow, ytrain) # training the model

prediction = lreg.predict_proba(xvalid_bow) # predicting on the validation set
prediction_int = prediction[:,1] >= 0.3 # if prediction is greater than or equal to 0.3 than 1 else 0
prediction_int = prediction_int.astype(np.int)

f1_score(yvalid, prediction_int,average='micro') # calculating f1 score

test_pred = lreg.predict_proba(test3)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=0.8)
nb.fit(xtrain_bow, ytrain)

preds = nb.predict(xvalid_bow)
f1_score(yvalid, preds,average='micro') # calculating f1 score

test_pred = nb.predict(test3)

In [ ]:
set(test_pred.astype(np.int))

In [ ]:
# set(prediction_int)
# max(prediction[:,1])

In [ ]:
subfin = pd.DataFrame({'ID': test['ID'].values, 'MaterialType': test_pred})
subfin=subfin.reindex(columns=["ID","MaterialType"])
subfin.to_csv('submission.csv', index=False)

In [ ]:
subfin.head()

In [ ]:
subfin['MaterialType'].replace(0,"BOOK",inplace=True)
subfin['MaterialType'].replace(1,"CR",inplace=True)
subfin['MaterialType'].replace(2,"MIXED",inplace=True)
subfin['MaterialType'].replace(3,"MUSIC",inplace=True)
subfin['MaterialType'].replace(4,"SOUNDCASS",inplace=True)
subfin['MaterialType'].replace(5,"SOUNDDISC",inplace=True)
subfin['MaterialType'].replace(6,"VIDEOCASS",inplace=True)
subfin['MaterialType'].replace(7,"VIDEODISC",inplace=True)

In [ ]:
subfin.to_csv('submission.csv', index=False)

In [ ]:
# import the modules we'll need
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "submission.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
create_download_link(subfin)

## Flair

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

train['Publisher'] = le.fit_transform(train['Publisher'])
test['Publisher'] = le.transform(test['Publisher'])

train['Creator'] = le.fit_transform(train['Creator'])
test['Creator'] = le.transform(test['Creator'])

In [ ]:
set(train['Creator']) - set(test['Creator'])

In [ ]:
from flair.data import Sentence
# create a sentence #
sentence = Sentence('Awesomeness come from the core.')
# print the sentence to see what’s in it. #
print(Sentence.get_embedding)

In [ ]:
#extracting the tweet part#
text = train['combined'] 
 ## txt is a list of tweets ##
txt = text.tolist()
print(txt[:10])

In [ ]:
## Importing the Embeddings ##
from flair.embeddings import WordEmbeddings
from flair.embeddings import CharacterEmbeddings
from flair.embeddings import StackedEmbeddings
from flair.embeddings import FlairEmbeddings
from flair.embeddings import BertEmbeddings
from flair.embeddings import ELMoEmbeddings
from flair.embeddings import FlairEmbeddings

### Initialising embeddings (un-comment to use others) ###
#glove_embedding = WordEmbeddings('glove')
#character_embeddings = CharacterEmbeddings()
flair_forward  = FlairEmbeddings('news-forward-fast')
flair_backward = FlairEmbeddings('news-backward-fast')
#bert_embedding = BertEmbedding()
#elmo_embedding = ElmoEmbedding()

stacked_embeddings = StackedEmbeddings( embeddings = [ 
                                                       flair_forward, 
                                                       flair_backward
                                                      ])

In [ ]:
# create a sentence #
sentence = Sentence("Awesomeness come from the core.")
# embed words in sentence #
stacked_embeddings.embed(sentence)
for token in sentence:
      print(token.embedding)
# data type and size of embedding #
print(type(token.embedding))
# storing size (length) #
z = token.embedding.size()[0]

In [ ]:
from flair.embeddings import DocumentPoolEmbeddings
from tqdm import tqdm
### initialize the document embeddings, mode = mean ###
document_embeddings = DocumentPoolEmbeddings([flair_backward, flair_forward ])
s = torch.zeros(0,z)
# iterating Sentences #
for tweet in tqdm(txt):
        #print(tweet)
        sentence = Sentence(tweet)
        document_embeddings.embed(sentence)
        # Storing Size of embedding #
        z = sentence.embedding.size()[0]
        # Adding Document embeddings to list #
        s = torch.cat((s, sentence.embedding.view(-1,z)),0)

In [ ]:
# s = torch.cat((s,torch.zeros(2048).view(-1,2048)),0)

In [ ]:
## tensor to numpy array ##
X = s.numpy()

In [ ]:
np.save('train_embed',np.column_stack((X, train['Checkouts'],train['PublicationYear'])))

In [ ]:
#extracting the tweet part#
text = test['combined'] 
txt = text.tolist()

from flair.embeddings import WordEmbeddings
from flair.embeddings import CharacterEmbeddings
from flair.embeddings import StackedEmbeddings
from flair.embeddings import FlairEmbeddings
from flair.embeddings import BertEmbeddings
from flair.embeddings import ELMoEmbeddings
from flair.embeddings import FlairEmbeddings

flair_forward  = FlairEmbeddings('news-forward-fast')
flair_backward = FlairEmbeddings('news-backward-fast')

stacked_embeddings = StackedEmbeddings( embeddings = [ flair_forward, flair_backward])

# create a sentence #
sentence = Sentence("Analytics Vidhya blogs are Awesome.")
# embed words in sentence #
stacked_embeddings.embed(sentence)
for token in sentence:
      print(token.embedding)
# storing size (length) #
z = token.embedding.size()[0]

from flair.embeddings import DocumentPoolEmbeddings
from tqdm import tqdm
### initialize the document embeddings, mode = mean ###
document_embeddings = DocumentPoolEmbeddings([flair_backward,flair_forward])
s = torch.zeros(0,z)
# iterating Sentences #
for tweet in tqdm(txt):
        sentence = Sentence(tweet)
        document_embeddings.embed(sentence)
        # Storing Size of embedding #
        z = sentence.embedding.size()[0]
        # Adding Document embeddings to list #
        s = torch.cat((s, sentence.embedding.view(-1,z)),0)
X1 = s.numpy()
np.save('test_embed',np.column_stack((X1, test['Checkouts'],test['PublicationYear'])))

In [ ]:
train1 = np.column_stack((X, train['Checkouts'],train['PublicationYear']))
test1 = np.column_stack((X1, test['Checkouts'],test['PublicationYear']))

In [ ]:
target = train['MaterialType'].values

In [ ]:
target

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

### Splitting training set ###
x_train, x_valid, y_train, y_valid = train_test_split(train1, target,random_state=42,test_size=0.2)

### XGBoost compatible data ###
dtrain = xgb.DMatrix(x_train,y_train)         
dvalid = xgb.DMatrix(x_valid, label = y_valid)

### defining parameters ###
params = {
          'colsample': 0.9,
          'colsample_bytree': 0.5,
          'eta': 0.1,
          'max_depth': 8,
          'min_child_weight': 6,
          'objective': 'multi:softmax',
          'subsample': 0.9,
        'num_class':8
          }

### Training the model ###
xgb_model = xgb.train(
                      params,
                      dtrain,
                      feval= custom_eval,
                      num_boost_round= 1000,
                      maximize=True,
                      evals=[(dvalid, "Validation")],
                      early_stopping_rounds=30
                      )

In [ ]:
# params = {'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'multiclass',
#     'num_class':8,
#     'metric': 'custom_eval',
#     'learning_rate': 0.002296,
#     'max_depth': 7,
#     'num_leaves': 17,
#     'metric': ['multi_error'],
#     'feature_fraction': 0.4,
#     'bagging_fraction': 0.6,
#     'bagging_freq': 17}

import lightgbm as lgb
### Splitting training set ###
x_train, x_valid, y_train, y_valid = train_test_split(train1, target,random_state=42,test_size=0.2)
print(x_train.shape)
d_train = lgb.Dataset(x_train, label=y_train)
params = {}
# params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'multiclass'
params['num_class'] = 8
params['metric'] = ['multi_error']
# params['sub_feature'] = 0.5
# params['num_leaves'] = 10
# params['min_data'] = 50
# params['max_depth'] = 10
params['silent'] = 2
clf = lgb.train(params, d_train, 2000)

In [ ]:
'done'

In [ ]:
y_pred=clf.predict(test1)

In [ ]:
# y_pred.shape
set(np.argmax(y_pred,axis=1))

In [ ]:
def custom_eval(preds, dtrain):
    labels = dtrain.get_label().astype(np.int)
    preds = (preds >= 0.3).astype(np.int)
    return [('f1_score', f1_score(labels, preds, average=None))]

In [ ]:
# import tensorflow_hub as hub
# import tensorflow as tf

# elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [ ]:
# # just a random sentence
# x = ["Roasted ants are a popular snack in Columbia"]

# # Extract ELMo features 
# embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

# embeddings.shape

In [ ]:
# embeddings.shape

In [ ]:
# def elmo_vectors(x):
#   embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

#   with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     sess.run(tf.tables_initializer())
#     # return average of ELMo features
#     return sess.run(tf.reduce_mean(embeddings,1))

In [ ]:
# list_train = [train[i:i+100] for i in range(0,train.shape[0],100)]
# list_test = [test[i:i+100] for i in range(0,test.shape[0],100)]

In [ ]:
# %%time
# # Extract ELMo embeddings
# elmo_train = [elmo_vectors(x['Subjects']) for x in list_train]
# elmo_test = [elmo_vectors(x['Subjects']) for x in list_test]

### 2nd Problem

In [ ]:
nl_train = pd.read_csv('../input/nlp_datac2476d7/NLP_Data/train.csv')
nl_test = pd.read_csv('../input/nlp_datac2476d7/NLP_Data/test.csv')
nl_subm = pd.read_csv('../input/nlp_datac2476d7/NLP_Data/sample_submission.csv')

In [ ]:
nl_train.head()

In [ ]:
nl_test.index[nl_test['date']==' Jan 0, 0000']

### changing the wrong date values to default date

In [ ]:
nl_test['date'].replace('None'," Jan 1, 2000",inplace=True)
nl_test['date'].replace(' Nov 0, 0000'," Jan 1, 2000",inplace=True)
nl_test['date'].replace(' Jan 0, 0000'," Jan 1, 2000",inplace=True)

### create features like day,month & year from the date.

In [ ]:
from datetime import datetime
def convert_date(d):
    return datetime.strptime(d,' %b %d, %Y')
# datetime.strptime(nl_train['date'][0], ' %b %d, %Y').date().day
nl_train['day'] = nl_train['date'].apply(lambda x: convert_date(x).date().day)
nl_train['month'] = nl_train['date'].apply(lambda x: convert_date(x).date().month)
nl_train['year'] = nl_train['date'].apply(lambda x: convert_date(x).date().year)

nl_test['day'] = nl_test['date'].apply(lambda x: convert_date(x).date().day)
nl_test['month'] = nl_test['date'].apply(lambda x: convert_date(x).date().month)
nl_test['year'] = nl_test['date'].apply(lambda x: convert_date(x).date().year)

In [ ]:
pd.to_datetime(nl_train.date.str, format='%b %d,%Y', yearfirst=False)

### finding the frequency of target classes

In [ ]:
print('class count in numbers: ')
nl_train['overall'].value_counts()
print('percentage of class count : ')
nl_train['overall'].value_counts()/train.shape[0] * 100

In [ ]:
nl_train.nunique()
nl_test.nunique()

In [ ]:
# Number of NaNs in each rows
nl_train.isnull().sum(axis=0)
nl_test.isnull().sum(axis=0)

### label encode the text fields

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(nl_train['Place'])
nl_train['Place'] = le.transform(nl_train['Place'])
nl_test['Place'] = le.transform(nl_test['Place'])

In [ ]:
le = LabelEncoder()
le.fit(nl_train['status'])
nl_train['status'] = le.transform(nl_train['status'])
nl_test['status'] = le.transform(nl_test['status'])

In [ ]:
nl_test['negatives'].replace(np.nan,"",inplace=True)

nl_train['advice_to_mgmt'].replace(np.nan,"",inplace=True)
nl_test['advice_to_mgmt'].replace(np.nan,"",inplace=True)

nl_train['summary'].replace(np.nan,"",inplace=True)
nl_test['summary'].replace(np.nan,"",inplace=True)

### combine all the text column into one

In [ ]:
nl_train['combined'] = nl_train['summary'] + ' . ' + nl_train['positives'] + ' . ' + nl_train['negatives'] + ' . ' +  nl_train['advice_to_mgmt']
nl_test['combined'] = nl_test['summary'] + ' . ' + nl_test['positives'] + ' . ' + nl_test['negatives'] + ' . ' +  nl_test['advice_to_mgmt']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# TF-IDF feature matrix
train1 = tfidf_vectorizer.fit_transform(nl_train['combined'])
test1 = tfidf_vectorizer.fit_transform(nl_test['combined'])

from sklearn.feature_extraction.text import CountVectorizer
# create the transform
vectorizer = CountVectorizer(stop_words="english", analyzer='word', 
                            ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None)
# tokenize and build vocab
vectorizer.fit(np.hstack([nl_train['combined'].values,nl_test['combined'].values]))
print(train2.shape)
print(type(train2))
train2 = vectorizer.transform(nl_train['combined'])
test2 = vectorizer.transform(nl_test['combined'])

from scipy.sparse import hstack, csr_matrix,vstack
train3 = hstack((train1, train2))
test3 = hstack((test1, test2))

train3
test3

### naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=0.8)
# splitting data into training and validation set
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train3, nl_train['overall'], random_state=42, test_size=0.2)

nb.fit(xtrain_bow, ytrain)

preds = nb.predict(xvalid_bow)
f1_score(yvalid, preds,average='micro') # calculating f1 score
test_pred = nb.predict(test3)

### create more features

In [ ]:
%%time
def get_polarity(txt):
    return TextBlob(txt).sentiment.polarity
nl_train['summ_score'] = nl_train['summary'].apply(lambda x:get_polarity(x))
nl_train['pos_score'] = nl_train['positives'].apply(lambda x:get_polarity(x))
nl_train['neg_score'] = nl_train['negatives'].apply(lambda x:get_polarity(x))
nl_train['adv_score'] = nl_train['advice_to_mgmt'].apply(lambda x:get_polarity(x))

nl_test['summ_score'] = nl_test['summary'].apply(lambda x:get_polarity(x))
nl_test['pos_score'] = nl_test['positives'].apply(lambda x:get_polarity(x))
nl_test['neg_score'] = nl_test['negatives'].apply(lambda x:get_polarity(x))
nl_test['adv_score'] = nl_test['advice_to_mgmt'].apply(lambda x:get_polarity(x))

In [ ]:
nl_train['score_1'].replace(np.nan,nl_train['score_1'].mean(),inplace=True)

In [ ]:
column = ['Place','status','score_1','score_2','score_3','score_4','score_5','score_6','day','month','year']
#'summ_score','pos_score','neg_score','adv_score'

In [ ]:
nl_train['score_1'].replace(np.nan,nl_train['score_1'].mean(),inplace=True)
nl_train['score_2'].replace(np.nan,nl_train['score_2'].mean(),inplace=True)
nl_train['score_3'].replace(np.nan,nl_train['score_3'].mean(),inplace=True)
nl_train['score_4'].replace(np.nan,nl_train['score_4'].mean(),inplace=True)
nl_train['score_5'].replace(np.nan,nl_train['score_5'].mean(),inplace=True)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=0.8)

# splitting data into training and validation set
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(nl_train[column], nl_train['overall'], random_state=42, test_size=0.2)

nb.fit(xtrain_bow, ytrain)

preds = nb.predict(xvalid_bow)
f1_score(yvalid, preds,average='micro') # calculating f1 score
test_pred = nb.predict(nl_test[column])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# splitting data into training and validation set
# xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(nl_train[column], nl_train['overall'], random_state=42, test_size=0.2)
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train3, nl_train['overall'], random_state=42, test_size=0.2)

# lreg = LogisticRegression()
lreg = lgb.LGBMRegressor()
lreg.fit(xtrain_bow, ytrain) # training the model

prediction = lreg.predict(xvalid_bow) # predicting on the validation set
# prediction_int = prediction[:,1] >= 0.3 # if prediction is greater than or equal to 0.3 than 1 else 0
prediction_int = prediction.astype(np.int)

f1_score(yvalid, prediction_int,average='micro') # calculating f1 score

test_pred = lreg.predict(test3)

In [ ]:
set(test_pred.astype(np.int))

In [ ]:
subfin = pd.DataFrame({'ID': nl_test['ID'].values, 'overall': test_pred.astype(np.int)})
subfin=subfin.reindex(columns=["ID","overall"])
subfin.to_csv('submission.csv', index=False)

In [ ]:
# import the modules we'll need
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "submission.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
create_download_link(subfin)